In this notebook, we want to build a logistic regression classifier in Tensorflow for MNIST.

The logistic regression classifier is defined as: $y = sigmoid(Wx + b)$

In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import math

In [2]:
# Get the MNIST dataset
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# Step 0: Read in the data into the "/data/mnist" folder
mnist = input_data.read_data_sets('./data/mnist', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [4]:
# Step 1: Create placeholders to feed your inputs and labels into
# Each MNIST image has 28*28 = 784 pixels. So you can represent it as a 1x784 Tensor.
# There are 10 possible classes for each image, corresponding to digits 0-9.
# Name the input placeholder mnist_inputs and the labels placeholder mnist_labels
mnist_inputs = tf.placeholder("float", [None, 784], name="mnist_inputs_placeholder") 
mnist_labels = tf.placeholder("float", [None, 10], name='mnist_labels_placeholder')

In [5]:
# Step 2: Create Variables for the parameters of the model: the weights and the bias.
# Initialize the bias to a 0 tensor. (hint: tf.zeros)
# Initialize the weights with a random uniform distribution, with a max of 1 and a min of -1. (hint: tf.random_uniform)
# Be sure to think carefully about the shapes of these tensors.
W = tf.Variable(tf.random_uniform([784, 10], -1, 1), name="weights")
b = tf.Variable(tf.zeros(shape=[1, 10]), name="bias")

# Optional: Define a global_step variable for use in tensorboard
global_step = tf.Variable(0, name='global_step', trainable=False)

In [6]:
# Step 3: Build the model, stringing together your placeholders and variables to create
# two ops: one for the logits (output right before sigmoid), and one for the probability
# distribution generated from the model (output right after sigmoid/softmax operation).
# tf.nn.softmax may come in handy for generating the probabilities.

# Name the logits operation "logits", and the probability operation "predictions".
with tf.name_scope("model"):
    logits = tf.matmul(mnist_inputs, W) + b
    predictions = tf.nn.softmax(logits)

In [7]:
# Step 4: Define your loss function. Use the cross entropy loss function, and use tensorflow's
# built in "tf.nn.sofmtax_cross_entropy_with_logits(logits, labels)" function to get the xentropy
# of each instance in the batch. Then, get the average loss of the batch.
# Name the loss op "loss"
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=mnist_labels, name="xentropy"))

In [8]:
# Step 5: Define a function to get the accuracy of your model on this batch. Name it "accuracy"
with tf.name_scope("accuracy"):
    correct_preds = tf.equal(tf.argmax(predictions, 1), tf.argmax(mnist_labels, 1))
    num_correct_preds = tf.reduce_sum(tf.cast(correct_preds, "float"))
    accuracy = num_correct_preds / tf.cast(tf.shape(mnist_inputs)[0], "float")

In [9]:
# Step 6: Define an optimizer that you want to use, and create the training operation to 
# use the optimizer to minimize the loss. Name the training operation "train_op"
with tf.name_scope("train_op"):
    train_op = tf.train.AdamOptimizer().minimize(loss, global_step=global_step)

In [10]:
# Define summary ops for TensorBoard (optional). Name the summary op "summary_op".
with tf.name_scope("summaries"):
    tf.summary.scalar("loss", loss)
    tf.summary.scalar("accuracy", accuracy)
    summary_op = tf.summary.merge_all()

In [11]:
# Step 7: Create a session for the model to run in, and then set up a train loop
# to optimize the weights given the mnist data. Optionally, add tensorboard visualization too.
nb_train_examples = mnist.train.num_examples
batch_size = 128
nb_epochs = 30
batches_per_epoch = int(math.ceil(nb_train_examples/batch_size))
log_period = 250

with tf.Session() as sess:
    # Step 7.1 Initialize your Variables
    sess.run(tf.global_variables_initializer())
    # Set up tensorboard writer (optional)
    train_writer = tf.summary.FileWriter("./logs/train",
                                         sess.graph)
    for epoch in range(nb_epochs):
        epoch_total_loss = 0
        epoch_total_accuracy = 0
        
        for batch in range(batches_per_epoch):
            loop_global_step = sess.run(global_step) + 1
            batch_inputs, batch_labels = mnist.train.next_batch(batch_size)
            # Step 7.2 Get the batch loss, batch accuracy, and run the training op.
            # If the log period is up, write summaries to tensorboard.
            batch_loss, batch_acc, _ = sess.run([loss, accuracy, train_op], 
                                         feed_dict={mnist_inputs: batch_inputs, 
                                                    mnist_labels: batch_labels})
            if loop_global_step % log_period == 0:
                train_summary = sess.run(summary_op, 
                                         feed_dict={mnist_inputs: batch_inputs, 
                                                    mnist_labels: batch_labels})
                train_writer.add_summary(train_summary, loop_global_step)
                
            epoch_total_loss += batch_loss
            epoch_total_accuracy += batch_acc
        
        epoch_average_loss = epoch_total_loss / batches_per_epoch
        epoch_average_accuracy = epoch_total_accuracy / batches_per_epoch
        print("Epoch {} done! Average Train Loss: {}, Average Train Accuracy: {}".format(epoch, 
                                                                                 epoch_average_loss, 
                                                                                 epoch_average_accuracy))
    print("Finished {} epochs".format(nb_epochs))

Epoch 0 done! Average Train Loss: 3.4256333865398583, Average Train Accuracy: 0.3581031976744186
Epoch 1 done! Average Train Loss: 1.0017955978249395, Average Train Accuracy: 0.7183321220930232
Epoch 2 done! Average Train Loss: 0.6738615992457367, Average Train Accuracy: 0.8082303779069767
Epoch 3 done! Average Train Loss: 0.548311646248019, Average Train Accuracy: 0.8456213662790698
Epoch 4 done! Average Train Loss: 0.4821911367219548, Average Train Accuracy: 0.8640261627906977
Epoch 5 done! Average Train Loss: 0.4398189597005068, Average Train Accuracy: 0.877234738372093
Epoch 6 done! Average Train Loss: 0.4083902920747912, Average Train Accuracy: 0.8861555232558139
Epoch 7 done! Average Train Loss: 0.38565764475700465, Average Train Accuracy: 0.8916424418604652
Epoch 8 done! Average Train Loss: 0.3688659559502158, Average Train Accuracy: 0.8971838662790698
Epoch 9 done! Average Train Loss: 0.35390536715818005, Average Train Accuracy: 0.9012172965116279
Epoch 10 done! Average Train L